In [9]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from itertools import chain, combinations
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np

# Load the data
jan_dlq2 = pd.read_csv(r"C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\DL\channel_engage\data\jan_dlq2.csv")
events_df = pd.read_csv(r"C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\DL\channel_engage\data\events_df.csv")

#work
#filtered to speed up creation time
#filtered_accounts = ['1581216589''1250636031']
#jan_dlq2 = jan_dlq2[jan_dlq2['acct_ref_nb'].isin(filtered_accounts)]
#events_df = events_df[events_df['acct_ref_nb'].isin(filtered_accounts)]

#update emailday2 to email and ob_attempts to outbound in events_df
#work
#events_df['channel'] = events_df['channel'].replace(['EMAILDAY2'], 'EMAIL')
#events_df['channel'] = events_df['channel'].replace(['OB_ATTEMPTS'], 'OUTBOUND')
#drop acc_balance segment probability_score and bucket from events df because it's already in jan_dlq2
#work
#events_df = events_df.drop(['acc_balance_segment', 'probability_score', 'bucket'])
#drop duplicates in events_df
events_df = events_df.drop_duplicates()
#check on E_LETTER or E_LETTER
interaction_channels = ['EMAIL', 'TEXT', 'LETTER', 'E_LETTER', 'OUTBOUND'] #Maybe take out letter and E_LETTER because timing is off unless it's 15 day letter
#For Updating the interaction channels list if we want to retrain on a different set of interaction channels
drop_channels = ['LETTER', 'E_LETTER']
#drop the columns in events_df that are in the channel column in drop_channels
events_df = events_df[~events_df['channel'].isin(drop_channels)]
#new list of interaction_channels
interaction_channels = [x for x in interaction_channels if x not in drop_channels]

target_channels = ['INBOUND', 'PYMT', 'LOGIN', 'PROMISE', 'PYMT_PROG']
#For Updating the target channels list if we want to retrain on a different set of target channels
drop_channels = ['INBOUND','LOGIN'] 
#drop the columns in events_df that are in the channel column in drop_channels
events_df = events_df[~events_df['channel'].isin(drop_channels)]
#new list of target_channels
target_channels = [x for x in target_channels if x not in drop_channels]

#fix the date formattings so it's usable
#work
#jan_dlq2['src_data_dt'] = pd.to_datetime(jan_dlq2['src_data_dt'], format="%d%b%y:%H:%M:%S").dt.date
#events_df['date'] = pd.to_datetime(events_df["date"], format='%d%b%Y').dt.date
#not work
jan_dlq2['src_data_dt'] = pd.to_datetime(jan_dlq2['src_data_dt'], format="%d%b%y:%H:%M:%S").dt.date
events_df['date'] = pd.to_datetime(events_df["date"], format='%d-%b-%y').dt.date


#get month from cycle_dt and date so we can merge on month
#work
#jan_dlq2['month'] = pd.to_datetime(jan_dlq2["cycle_dt"], format='%m/%d/%Y').dt.to_period('M').dt.to_timestamp()
#events_df['month'] = pd.to_datetime(events_df["date"], format='%d%b%Y').dt.to_period('M').dt.to_timestamp()
#not work
jan_dlq2['month'] = pd.to_datetime(jan_dlq2["cycle_dt"], format='%m/%d/%Y').dt.to_period('M').dt.to_timestamp()
events_df['month'] = pd.to_datetime(events_df["date"], format='%Y-%m-%d').dt.to_period('M').dt.to_timestamp()

#review the data
print(jan_dlq2.head(), events_df, sep='\n\n')
print(jan_dlq2.shape, events_df.shape)

   acct_ref_nb  acct_balance  tot_delq_amt  min_delq_amt  probability_score  \
0       125001           250           100            25              0.534   
1       125001           250           100            25              0.534   
2       125001           250           100            25              0.534   
3       125001           250           100            25              0.534   
4       125001           250           100            25              0.534   

  src_data_dt  cycle_dt state  zip_code   dlq_30   dlq_60   dlq_90  \
0  2022-01-31  1/5/2022    KY     40288  current  current  current   
1  2022-01-31  1/5/2022    KY     40288  current  current  current   
2  2022-01-31  1/5/2022    KY     40288  current  current  current   
3  2022-01-31  1/5/2022    KY     40288  current  current  current   
4  2022-01-31  1/5/2022    KY     40288  current  current  current   

  exclusion_type       roll_30       roll_60       roll_90 segment  bucket  \
0       elgiible  roll for

In [10]:
#preprocess by concatenating events_df into a row per date, acct_ref_nb, and target_channel type.
#^^ this allows the volume to stay correct and us to be able to assess target channel's by their combinations
# One-hot encode the 'channel' column
df_concatenate = pd.get_dummies(events_df, columns=['channel'], prefix='', prefix_sep='')

# Aggregate interaction channels by 'acct_ref_nb' and 'date'
df_agg_concatenate = df_concatenate.drop(columns=target_channels).groupby(['acct_ref_nb', 'date', 'month']).sum().reset_index()

# Aggregate target channels for each 'acct_ref_nb' and 'date'
df_targets_concatenate = df_concatenate.groupby(['acct_ref_nb', 'date', 'month'])[target_channels].sum().reset_index()
df_targets_concatenate['target_channel'] = df_targets_concatenate[target_channels].apply(lambda x: '_'.join(x.index[x > 0]), axis=1)

# Merge the aggregated target channels back into the feature dataframe
df_agg_concatenate = df_agg_concatenate.merge(df_targets_concatenate[['acct_ref_nb', 'date', 'month', 'target_channel']], on=['acct_ref_nb', 'date', 'month'], how='left')

#replace blanks with 'NO_PYMT'
df_agg_concatenate['target_channel'] = df_agg_concatenate['target_channel'].replace([''], 'NO_PYMT')

# Replace non-blanks with 'PYMT' except for 'NO_PYMT'
mask = (df_agg_concatenate['target_channel'] != '') & (df_agg_concatenate['target_channel'] != 'NO_PYMT')
df_agg_concatenate.loc[mask, 'target_channel'] = 'PYMT'

print(df_agg_concatenate.head(), df_agg_concatenate.shape, sep='\n\n')

#export to csv
df_agg_concatenate.to_csv(r"C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\DL\channel_engage\data\df_agg_concatenate.csv")

   acct_ref_nb        date      month  EMAIL  OUTBOUND  TEXT target_channel
0       125001  2022-01-31 2022-01-01      1         1     1           PYMT
1       125001  2022-02-01 2022-02-01      1         1     1           PYMT
2       125001  2022-02-02 2022-02-01      1         0     0           PYMT
3       125001  2022-02-03 2022-02-01      1         0     0           PYMT
4       125001  2022-02-04 2022-02-01      1         1     0           PYMT

(51, 7)


In [11]:
#change everything in the target_channel not named 'NO_PYMT' to 'PYMT'

print(df_agg_concatenate.head())
#print target_channel value counts
print(df_agg_concatenate['target_channel'].value_counts())

#get unique jan_dlq2 rows to get categorical variables for the model
jan_dlq_no_dupes = jan_dlq2.drop_duplicates()

unique_keys_df1 = df_agg_concatenate[['acct_ref_nb', 'month']].drop_duplicates()
unique_keys_df2 = jan_dlq_no_dupes[['acct_ref_nb', 'month']].drop_duplicates()

merged_keys = pd.merge(unique_keys_df1, unique_keys_df2, on=['acct_ref_nb', 'month'], how='inner')
missing_keys_df1 = unique_keys_df1[~unique_keys_df1.isin(merged_keys)].dropna()
missing_keys_df2 = unique_keys_df2[~unique_keys_df2.isin(merged_keys)].dropna()

print("Missing keys in df_agg_concatenate:", len(missing_keys_df1))
print("Missing keys in jan_dlq_no_dupes:", len(missing_keys_df2))

duplicates_df1 = df_agg_concatenate[df_agg_concatenate.duplicated(subset=['acct_ref_nb', 'month'])]
duplicates_df2 = jan_dlq_no_dupes[jan_dlq_no_dupes.duplicated(subset=['acct_ref_nb', 'month'])]

print("Duplicates in df_agg_concatenate:", len(duplicates_df1))
print("Duplicates in jan_dlq_no_dupes:", len(duplicates_df2))

nan_values_df1 = df_agg_concatenate[df_agg_concatenate[['acct_ref_nb', 'month']].isnull().any(axis=1)]
nan_values_df2 = jan_dlq_no_dupes[jan_dlq_no_dupes[['acct_ref_nb', 'month']].isnull().any(axis=1)]

print("NaN values in df_agg_concatenate:", len(nan_values_df1))
print("NaN values in jan_dlq_no_dupes:", len(nan_values_df2))



   acct_ref_nb        date      month  EMAIL  OUTBOUND  TEXT target_channel
0       125001  2022-01-31 2022-01-01      1         1     1           PYMT
1       125001  2022-02-01 2022-02-01      1         1     1           PYMT
2       125001  2022-02-02 2022-02-01      1         0     0           PYMT
3       125001  2022-02-03 2022-02-01      1         0     0           PYMT
4       125001  2022-02-04 2022-02-01      1         1     0           PYMT
NO_PYMT    33
PYMT       18
Name: target_channel, dtype: int64
Missing keys in df_agg_concatenate: 4
Missing keys in jan_dlq_no_dupes: 5
Duplicates in df_agg_concatenate: 45
Duplicates in jan_dlq_no_dupes: 13
NaN values in df_agg_concatenate: 0
NaN values in jan_dlq_no_dupes: 0


In [12]:


#print(jan_dlq_no_dupes)
#print(jan_dlq_no_dupes.shape)

# Merge data on acct_ref_nb and month
data = pd.merge(df_agg_concatenate, jan_dlq_no_dupes, on=["acct_ref_nb", "month"], how="left") #events_df_agg and jan_dlq_no_dupes for when this is implemented, check the data names for the ROC at the end
print(data.head())
print(data.shape)

#print datas columns
print(data.columns)
#how to select just the buckets column to show
print(data['bucket'].head())

   acct_ref_nb        date      month  EMAIL  OUTBOUND  TEXT target_channel  \
0       125001  2022-01-31 2022-01-01      1         1     1           PYMT   
1       125001  2022-02-01 2022-02-01      1         1     1           PYMT   
2       125001  2022-02-01 2022-02-01      1         1     1           PYMT   
3       125001  2022-02-01 2022-02-01      1         1     1           PYMT   
4       125001  2022-02-01 2022-02-01      1         1     1           PYMT   

   acct_balance  tot_delq_amt  min_delq_amt  probability_score src_data_dt  \
0           250           100            25              0.534  2022-01-31   
1           500           250            50              0.734  2022-02-01   
2           500           250            50              0.434  2022-02-02   
3           500           250            50              0.334  2022-02-02   
4           500           250            50              0.120  2022-02-03   

   cycle_dt state  zip_code   dlq_30   dlq_60   dlq_90 e

In [13]:

# Define bucket boundaries and labels for different features
bucket_definitions = {
    'probability_score': {
        'bins': [0, 0.2, 0.4, 0.6, 0.8, 1.0],
        'labels': ['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1.0'],
        'column_name': 'long_term_risk_score'
    },
    'acct_balance': {
        'bins': [0, 100, 500, 1000, 5000, 10000, 50000, 100000],
        'labels': ['0-100', '100-500', '500-1000', '1000-5000', '5000-10000', '10000-50000', '50000-100000'],  # Added one label
        'column_name': 'acct_balance_segments'
    },
    'tot_delq_amt': {
        'bins': [0, 100, 500, 1000, 5000, 10000, 50000, 100000],
        'labels': ['0-100', '100-500', '500-1000', '1000-5000', '5000-10000', '10000-50000', '50000-100000'],  # Added one label
        'column_name': 'tot_delq_balance_segments'
    },
    'min_delq_amt': {
        'bins': [0, 100, 500, 1000, 5000, 10000, 50000, 100000],
        'labels': ['0-100', '100-500', '500-1000', '1000-5000', '5000-10000', '10000-50000', '50000-100000'],  # Added one label
        'column_name': 'min_delq_balance_segments'
    }
}

# Cut data into buckets
for feature, definitions in bucket_definitions.items():
    column_name = definitions['column_name']
    data[column_name] = pd.cut(data[feature], bins=definitions['bins'], labels=definitions['labels'], include_lowest=True, right=False)
print(data.head())
# export pre-preprocessor data to csv
#data.to_csv(r"C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\DL\channel_engage\data\prepreprocessed_data.csv", index=False)

# If you're going to use models like linear or logistic regression, use the following lines to one-hot encode
#columns_to_onehot = [defi['column_name'] for defi in bucket_definitions.values()]
#data = pd.get_dummies(data, columns=columns_to_onehot, prefix='', prefix_sep='')

# Change date to year-month-day
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

#print(data.head())

   acct_ref_nb        date      month  EMAIL  OUTBOUND  TEXT target_channel  \
0       125001  2022-01-31 2022-01-01      1         1     1           PYMT   
1       125001  2022-02-01 2022-02-01      1         1     1           PYMT   
2       125001  2022-02-01 2022-02-01      1         1     1           PYMT   
3       125001  2022-02-01 2022-02-01      1         1     1           PYMT   
4       125001  2022-02-01 2022-02-01      1         1     1           PYMT   

   acct_balance  tot_delq_amt  min_delq_amt  probability_score src_data_dt  \
0           250           100            25              0.534  2022-01-31   
1           500           250            50              0.734  2022-02-01   
2           500           250            50              0.434  2022-02-02   
3           500           250            50              0.334  2022-02-02   
4           500           250            50              0.120  2022-02-03   

   cycle_dt state  zip_code   dlq_30   dlq_60   dlq_90 e

In [14]:
#check for nulls step: will say if there are nulls in the data
if data.isnull().values.any():
    print("There are nulls in the data")
#print(data.isnull().sum())

#drop nulls
data = data.dropna()
print(data.shape)

#view the dataframe by interaction channels to understand the weight of each channel

# Reshape the dataframe using melt to have interaction channels in one column
melted_data = pd.melt(data, id_vars=['target_channel'], value_vars=interaction_channels, var_name='interaction_channel', value_name='value')

# Filter only rows where value is 1 (i.e., the interaction happened)
filtered_melted_data = melted_data[melted_data['value'] == 1]

# Create the contingency table
contingency_table = pd.crosstab(filtered_melted_data['interaction_channel'], filtered_melted_data['target_channel'])

print(contingency_table)

(259, 30)
target_channel       NO_PYMT  PYMT
interaction_channel               
EMAIL                    176    83
OUTBOUND                 128    51
TEXT                     128    35


In [15]:
#Preprocessing 
# Drop any non-essential columns
data = data.drop(columns=['cycle_dt', 'src_data_dt', 'date','acct_ref_nb', 'zip_code', 'state',
                            'acct_balance', 'tot_delq_amt', 'min_delq_amt', 'probability_score']) #account number is not needed because identifier is src_data_dt

print(data.head())
print(data.shape)

# export pre-preprocessor data to csv
data.to_csv(r"C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\DL\channel_engage\data\preprocessed_data.csv", index=False)

# Define feature lists
numerical_features = [ 'year', 'month', 'day', 'EMAIL', 'TEXT', 'OUTBOUND']
categorical_features = [col for col in data.columns if col not in numerical_features + [ 'target_channel']] #'interaction_channel' is needed in here while target_channel is the y variable

# Define transformers
numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

# Define X and y
X = data.drop(columns=['target_channel'])
print(X.head())
# Use LabelEncoder to encode target_channel
label_encoder = LabelEncoder()
y = data['target_channel']
y_encoded = label_encoder.fit_transform(y)
#print(y_encoded)

# Preprocess the data
X = preprocessor.fit_transform(X)
#print(X)

   month  EMAIL  OUTBOUND  TEXT target_channel   dlq_30   dlq_60   dlq_90  \
0      1      1         1     1           PYMT  current  current  current   
1      2      1         1     1           PYMT  current  current  current   
2      2      1         1     1           PYMT  current  current  current   
3      2      1         1     1           PYMT  current  current  current   
4      2      1         1     1           PYMT  current  current  current   

  exclusion_type        roll_30        roll_60        roll_90 segment  bucket  \
0       elgiible   roll forward   roll forward   roll forward    cure       1   
1       elgiible  roll backward  roll backward  roll backward    cure       1   
2       elgiible  roll backward  roll backward  roll backward    cure       1   
3       elgiible  roll backward  roll backward  roll backward    cure       1   
4       elgiible   roll forward  roll backward  roll backward    cure       1   

  long_term_risk_score acct_balance_segments tot_d

In [16]:
#Neural Network Model: Splitting the data into train, validation, and test sets
# If X is a sparse matrix, convert to a dense matrix
if hasattr(X, "todense"):
    X = X.todense()

X = np.asarray(X)

# Split the data
X_train, X_temp, y_train_encoded, y_temp_encoded = train_test_split(X, y_encoded, test_size=0.4, stratify=y_encoded) # can add stratify=y_encoded to make sure the split is even
X_val, X_test, y_val_encoded, y_test_encoded = train_test_split(X_temp, y_temp_encoded, test_size=0.5, stratify=y_temp_encoded) # can add stratify=y_encoded to make sure the split is even

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train_encoded)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.LongTensor(y_val_encoded)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test_encoded)

# Convert to PyTorch datasets and create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



print(X_train_tensor.shape, y_train_tensor.shape, X_val_tensor.shape, y_val_tensor.shape, X_test_tensor.shape, y_test_tensor.shape)
print(data['target_channel'].value_counts())


torch.Size([155, 30]) torch.Size([155]) torch.Size([52, 30]) torch.Size([52]) torch.Size([52, 30]) torch.Size([52])
NO_PYMT    176
PYMT        83
Name: target_channel, dtype: int64
